In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

# Ruta a tu ChromeDriver
driver_path = "C:/Users/Pedro/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"

# Configura el servicio de Selenium
service = Service(driver_path)

# Inicia el navegador con el servicio configurado
driver = webdriver.Chrome(service=service)

# URL de la página principal
url = "https://www.holaislascanarias.com/playas/"

# Abre la página con Selenium
driver.get(url)
time.sleep(5)  # Espera a que se cargue el contenido dinámico

# Extrae el HTML renderizado
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Lista para almacenar datos de todas las páginas
data = []

# Función para procesar las playas en la página actual
def procesar_playas(soup):
    playas = soup.find_all("div", class_="wrapper-item")
    for playa in playas:
        try:
            title_tag = playa.find("h3", class_="field__item")
            title = title_tag.get_text(strip=True) if title_tag else "Sin título"

            link_tag = playa.find("a", href=True)
            relative_link = link_tag["href"] if link_tag else None
            full_link = f"https://www.holaislascanarias.com{relative_link}" if relative_link else "Sin enlace"

            island_tag = playa.find("p")
            island = island_tag.get_text(strip=True) if island_tag else "Sin isla"

            img_tag = playa.find("img")
            img_src = img_tag["src"] if img_tag else "Sin imagen"
            full_img_src = f"https://www.holaislascanarias.com{img_src}" if img_src.startswith("/sites/") else img_src

            data.append({
                "title": title,
                "island": island,
                "link": full_link,
                "image": full_img_src
            })

        except Exception as e:
            print(f"Error procesando una playa: {e}")

# Procesa la página inicial
procesar_playas(soup)

# Itera por las páginas siguientes
while True:
    # Encuentra el botón de siguiente página dentro de <li class="next">
    next_button = soup.find("li", class_="next")
    if next_button:
        # Extrae el enlace dentro de <a>
        link_tag = next_button.find("a", href=True)
        if link_tag:
            next_page_url = link_tag["href"]
            print(f"Navegando a la siguiente página: {next_page_url}")
            
            # Carga la nueva página
            driver.get(next_page_url)
            time.sleep(5)
            
            # Extrae el HTML de la nueva página
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            
            # Procesa las playas en la nueva página
            procesar_playas(soup)
        else:
            break  # No hay más enlaces, termina el bucle
    else:
        break  # No hay más páginas, termina el bucle


# Cierra el navegador
driver.quit()

# Muestra los datos extraídos
for item in data:
    print(item)


Navegando a la siguiente página: https://www.holaislascanarias.com/playas/?resource_type=b_playas&page=1
Navegando a la siguiente página: https://www.holaislascanarias.com/playas/?resource_type=b_playas&page=2
Navegando a la siguiente página: https://www.holaislascanarias.com/playas/?resource_type=b_playas&page=3
Navegando a la siguiente página: https://www.holaislascanarias.com/playas/?resource_type=b_playas&page=4
Navegando a la siguiente página: https://www.holaislascanarias.com/playas/?resource_type=b_playas&page=5
Navegando a la siguiente página: https://www.holaislascanarias.com/playas/?resource_type=b_playas&page=6
Navegando a la siguiente página: https://www.holaislascanarias.com/playas/?resource_type=b_playas&page=7
Navegando a la siguiente página: https://www.holaislascanarias.com/playas/?resource_type=b_playas&page=8
Navegando a la siguiente página: https://www.holaislascanarias.com/playas/?resource_type=b_playas&page=9
Navegando a la siguiente página: https://www.holaislasc

In [35]:
for item in data:
    if item['link'].startswith('/'):
        item['link'] = 'https://www.holaislascanarias.com/playas' + item['link']

In [37]:
for item in data:
    if '/playas' in item['link']:
        item['link'] = item['link'].replace('/playas', '', 1)

In [39]:
import json

# Define the output file path
output_file = "data.jsonl"

# Write the data to a JSONL file
with open(output_file, "w", encoding="utf-8") as file:
    for item in data:
        file.write(json.dumps(item) + "\n")

In [47]:
from bs4 import BeautifulSoup
import requests
import json

# Cargar los datos existentes
with open("data.jsonl", "r") as file:
    beaches = [json.loads(line) for line in file]

# Lista para almacenar playas con enlaces no válidos
invalid_links = []

for beach in beaches:
    try:
        # Verificar si el enlace es válido
        if not beach["link"].startswith("https://www.holaislascanarias.com"):
            print(f"Enlace no válido para {beach['title']}: {beach['link']}")
            invalid_links.append(beach["title"])
            continue  # Saltar esta playa y seguir con la siguiente

        # Solicitar la página de la playa
        response = requests.get(beach["link"])
        soup = BeautifulSoup(response.content, "html.parser")

        # Buscar títulos y descripciones en 'texts-wrapper'
        titles_tags = soup.select("div.texts-wrapper h3.title")
        description_tags = soup.select("div.texts-wrapper p.description")

        # Extraer títulos y descripciones como pares
        titles = [tag.get_text(strip=True) for tag in titles_tags]
        descriptions = [tag.get_text(strip=True) for tag in description_tags]

        # Emparejar títulos y descripciones
        paired_descriptions = [{"title": t, "description": d} for t, d in zip(titles, descriptions)]

        beach["details"] = paired_descriptions if paired_descriptions else "Sin detalles"
        print(f"Detalles extraídos para {beach['title']}")
    except Exception as e:
        print(f"Error extrayendo detalles para {beach['title']}: {e}")
        beach["details"] = "Error al obtener detalles"
        continue

# Guardar los datos actualizados
with open("beaches_updated.jsonl", "w") as file:
    for beach in beaches:
        file.write(json.dumps(beach, ensure_ascii=False) + "\n")

# Guardar la lista de playas con enlaces no válidos
with open("invalid_links.json", "w") as file:
    json.dump(invalid_links, file, ensure_ascii=False, indent=4)

print("Detalles extraídos y guardados en 'beaches_updated.jsonl'")
print("Enlaces no válidos guardados en 'invalid_links.json'")


Detalles extraídos para Bajamar
Detalles extraídos para Playa La Arena
Detalles extraídos para Playa Grande (Puerto del Carmen)
Detalles extraídos para Caleta de Mero
Detalles extraídos para Playa de Torviscas
Detalles extraídos para Playa Puerto de La Estaca
Detalles extraídos para Grandes Playas de Corralejo
Detalles extraídos para Playa de Chinguarime
Detalles extraídos para Playa de Punta Larga
Detalles extraídos para Playa El Cabrón
Detalles extraídos para Echentive
Detalles extraídos para Antequera
Detalles extraídos para Playa de Nogales
Detalles extraídos para Las Burras
Detalles extraídos para Playa de Ajuy
Detalles extraídos para El Poris
Detalles extraídos para La Garita
Detalles extraídos para Playa de La Negra
Detalles extraídos para Playa de Ereses
Detalles extraídos para Playa de San Marcos
Detalles extraídos para Charco Verde
Detalles extraídos para Anfi del Mar
Detalles extraídos para Esquinzo
Detalles extraídos para Playa de Famara
Detalles extraídos para Playa de San

UnicodeEncodeError: 'charmap' codec can't encode character '\u25cf' in position 1223: character maps to <undefined>

In [48]:
# Guardar los datos actualizados
with open("beaches_updated.jsonl", "w", encoding="utf-8") as file:
    for beach in beaches:
        file.write(json.dumps(beach, ensure_ascii=False) + "\n")

# Guardar la lista de playas con enlaces no válidos
with open("invalid_links.json", "w", encoding="utf-8") as file:
    json.dump(invalid_links, file, ensure_ascii=False, indent=4)

print("Detalles extraídos y guardados en 'beaches_updated.jsonl'")
print("Enlaces no válidos guardados en 'invalid_links.json'")

Detalles extraídos y guardados en 'beaches_updated.jsonl'
Enlaces no válidos guardados en 'invalid_links.json'


In [51]:
#Quitamos las playas que no tienen detalles
beaches_with_details = [beach for beach in beaches if beach.get("details")]

# Editamos el archivo de playas actualizado
with open("beaches_updated.jsonl", "w", encoding="utf-8") as file:
    for beach in beaches_with_details:
        file.write(json.dumps(beach, ensure_ascii=False) + "\n")

In [3]:
import json
from langchain_ollama import OllamaLLM as Ollama


In [ ]:

# Cargar datos de las playas
with open("beaches_updated.jsonl", "r", encoding="utf-8") as file:
    beaches = [json.loads(line) for line in file]

# Función para generar preguntas y respuestas con Llama 3.2
def generate_qa_with_llama(description, beach_name):
    prompt = f"""
    Genera cinco preguntas relevantes basadas en la siguiente descripción de una playa, junto con sus respuestas. 
    Incluye preguntas sobre ubicación, actividades, características y otros detalles interesantes.

    Nombre de la playa: {beach_name}
    Descripción: {description}

    Salida esperada, no añadas NADA más ni al inicio ni al final, solo las preguntas y respuestas:
    - Pregunta 1: <Pregunta>
      Respuesta 1: <Respuesta>
    - Pregunta 2: <Pregunta>
      Respuesta 2: <Respuesta>
    """
    try:
        model = Ollama(model="llama3.2")
        response_text = model.invoke(prompt)
        print(f"Generando preguntas y respuestas para {beach_name}...")
        return response_text    
    except Exception as e:
        print(f"Error conectando con Llama 3.2: {e}")
        return None




In [5]:
# Generar preguntas y respuestas para cada playa
qa_pairs = []
for beach in beaches:
    try:
        if "details" in beach and isinstance(beach["details"], list):
            description = " ".join(
                [f"{detail['title']}: {detail['description']}" for detail in beach["details"]]
            )
            qa_output = generate_qa_with_llama(description, beach["title"])
            if qa_output:
                qa_pairs.append({"beach": beach["title"], "qa": qa_output})
        else:
            print(f"El campo 'details' no es una lista en la playa: {beach['title']}")
    except Exception as e:
        print(f"Error procesando la playa {beach['title']}: {e}")
# Guardar los resultados en JSONL
with open("llama_generated_qa.jsonl", "w", encoding="utf-8") as file:
    for qa in qa_pairs:
        file.write(json.dumps(qa, ensure_ascii=False) + "\n")

print("Pares de preguntas y respuestas generados con Llama 3.2.")


Generando preguntas y respuestas para Bajamar...
Generando preguntas y respuestas para Playa La Arena...
Generando preguntas y respuestas para Playa Grande (Puerto del Carmen)...
Generando preguntas y respuestas para Caleta de Mero...
Generando preguntas y respuestas para Playa de Torviscas...
Generando preguntas y respuestas para Playa Puerto de La Estaca...
Generando preguntas y respuestas para Grandes Playas de Corralejo...
Generando preguntas y respuestas para Playa de Chinguarime...
Generando preguntas y respuestas para Playa de Punta Larga...
Generando preguntas y respuestas para Playa El Cabrón...
Generando preguntas y respuestas para Echentive...
Generando preguntas y respuestas para Antequera...
Generando preguntas y respuestas para Playa de Nogales...
Generando preguntas y respuestas para Las Burras...
Generando preguntas y respuestas para Playa de Ajuy...
Generando preguntas y respuestas para El Poris...
Generando preguntas y respuestas para La Garita...
Generando preguntas 

In [12]:
import json

def process_jsonl(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        for line_number, line in enumerate(infile, start=1):
            try:
                # Cargar la línea como JSON
                data = json.loads(line)

                # Suponiendo que el texto está en la clave 'text'
                text = data.get("qa", "")
                if not text.strip():
                    print(f"Línea {line_number}: 'text' está vacío o no existe.")
                    continue

                # Dividir por saltos de línea y eliminar espacios vacíos
                parts = [part.strip() for part in text.split('\n') if part.strip()]

                # Crear pares prompt-answer
                for i in range(0, len(parts), 2):
                    prompt = parts[i]
                    answer = parts[i + 1] if i + 1 < len(parts) else ""
                    json.dump({"prompt": prompt, "answer": answer}, outfile, ensure_ascii=False)
                    outfile.write('\n')

            except json.JSONDecodeError as e:
                print(f"Línea {line_number}: Error decodificando JSON: {e}")
            except Exception as e:
                print(f"Línea {line_number}: Error inesperado: {e}")

    print("Procesamiento completado. Revisa el archivo de salida.")

if __name__ == "__main__":
    input_file = "llama_generated_qa.jsonl"  # Reemplázalo con el nombre de tu archivo de entrada
    output_file = "output.jsonl"  # Reemplázalo con el nombre del archivo de salida
    process_jsonl(input_file, output_file)


Procesamiento completado. Revisa el archivo de salida.
